# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.count()
# city_data_df.loc[city_data_df['Country'].isna()]

City_ID       581
City          581
Lat           581
Lng           581
Max Temp      581
Humidity      581
Cloudiness    581
Wind Speed    581
Country       579
Date          581
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    # scale = 0.01,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp']<=27) & 
                                 (city_data_df['Max Temp']>21) & 
                                 (city_data_df['Wind Speed']<4.5) &
                                 (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
map_plot = ideal_city_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    # scale = 0.01,
    color = "City"
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,['Hotel Name']] = pd.Series()

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,port elizabeth,ZA,-33.9180,25.5701,33,NaN
62,kristiansund,NO,63.1115,7.7320,73,NaN
220,laguna,US,38.4210,-121.4238,58,NaN
259,crane,US,31.3974,-102.3501,45,NaN
267,durant,US,33.9940,-96.3708,70,NaN
350,arkhangel'sk,RU,64.5401,40.5433,40,NaN
390,arraias,BR,-12.9288,-46.9437,40,NaN
402,xai-xai,MZ,-25.0519,33.6442,46,NaN
455,kiwira,TZ,-9.1667,33.5333,27,NaN
484,hammerfest,NO,70.6634,23.6821,38,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "filter":"",
    "bias":"",
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index,'Lng']
    lat = hotel_df.loc[index,'Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
kristiansund - nearest hotel: Comfort Hotel Fosna
laguna - nearest hotel: Holiday Inn Express & Suites
crane - nearest hotel: No hotel found
durant - nearest hotel: Quality Inn
arkhangel'sk - nearest hotel: Серафима
arraias - nearest hotel: Hotel Japão
xai-xai - nearest hotel: No hotel found
kiwira - nearest hotel: New land hotel
hammerfest - nearest hotel: Thon Hotel Hammerfest
pontes e lacerda - nearest hotel: Hotel Avenida
santa ana - nearest hotel: Holiday Inn Express
hays - nearest hotel: Fall Creek Inn
portales - nearest hotel: Best Western Plus


,City,Country,Lat,Lng,Humidity,Hotel Name
23,port elizabeth,ZA,-33.9180,25.5701,33,Waterford Hotel
62,kristiansund,NO,63.1115,7.7320,73,Comfort Hotel Fosna
220,laguna,US,38.4210,-121.4238,58,Holiday Inn Express & Suites
259,crane,US,31.3974,-102.3501,45,No hotel found
267,durant,US,33.9940,-96.3708,70,Quality Inn
350,arkhangel'sk,RU,64.5401,40.5433,40,Серафима
390,arraias,BR,-12.9288,-46.9437,40,Hotel Japão
402,xai-xai,MZ,-25.0519,33.6442,46,No hotel found
455,kiwira,TZ,-9.1667,33.5333,27,New land hotel
484,hammerfest,NO,70.6634,23.6821,38,Thon Hotel Hammerfest


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    # scale = 0.01,
    color = "City",
    hover_cols = ['Hotel Name', 'Country']
)
# Display the map
map_plot
# hotel_df.hvplot.point

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)